This notebook lays out the initial set of analyses performed by RP using multidimensional item response theory (MIRT) on a subset of the UH2 discovery sample.  

For the analyses of the survey data, we have discussed two potential approaches, to be pursued in parallel.  One is to examine the survey results using the standard subscales associated with each survey; that approach is not addressed here.  The second, which I focus on here, is to estimate a set of factors without regard to the inventories from which the surveys come, in essence treating them as a "bag of items".  

I started with data for 158 subjects from the discovery sample on all of the survey items (limited to the surveys that have reasonable ordinal scales).  This gave 507 items.  In order to make these a bit more tractable to deal with, I extracted the items separately for each survey and saved them to separate files, along with the metadata for each survey.  These are all saved according to the (still incomplete) [BIDS standard](http://bids.neuroimaging.io) for phenotypic data.  




In [5]:
# first, set up our imports and configuration

import os,glob,pickle,sys
import json
import importlib

import pandas
import networkx as nx
import numpy
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display, HTML
pandas.set_option('max_colwidth', 100)

from show_image import showPDF

%load_ext rpy2.ipython

%matplotlib inline

# this is kludgey but it works
sys.path.append('../utils')

from utils import get_info
dataset='Discovery_9-26-16'
basedir=get_info('base_directory')
derived_dir=os.path.join(basedir,'data/Derived_Data/%s'%dataset)





The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [6]:
data,survey_metadata = pickle.load(open(os.path.join(derived_dir,'surveydata.pkl'),'rb'))


In [8]:
print('Processed the following surveys for %d subjects:'%len(data.worker.unique()))
print('data stored in %s'%derived_dir)
print('')

for k in survey_metadata.keys():
    surveyinfo.append([k.replace('_survey',''),len(survey_metadata[k])-1])
df=pandas.DataFrame(surveyinfo,columns=['SurveyName','NumberOfItems'])
display(df)
print(df.NumberOfItems.sum(),'items total')


Processed the following surveys for 200 subjects:
data stored in /Users/poldrack/code/Self_Regulation_Ontology/data/Derived_Data/Discovery_9-26-16



,SurveyName,NumberOfItems
0,ten_item_personality,10
1,upps_impulsivity,59
2,bis_bas,24
3,dickman,23
4,brief_self_control,13
5,self_regulation,30
6,leisure_time_activity,1
7,bis11,30
8,erq,10
9,impulsive_venture,34


1012 items total


Create a file containing the item ids and text, for use in later R analyses.

In [9]:

with open('all_survey_metadata.json', encoding='utf-8') as data_file:
        md = json.loads(data_file.read())

all_metadata={}
for measure in md.keys():
    for k in md[measure].keys():
        all_metadata[k]=md[measure][k]

with open('variable_key.txt','w') as f:
    for k in all_metadata.keys():
        f.write('%s\t%s\n'%(k,all_metadata[k]['Description']))

FileNotFoundError: [Errno 2] No such file or directory: 'all_survey_metadata.json'

### MIRT analyses

I then used the R mirt package to run exploratory multidimensional IRT analyses, using a 2PL model with the Metropolis-Hastings Robbins-Monro (MHRM) algorithm so that I could successfully fit the larger models.  In the example shown below, I fit models with dimensionalities from 1 to 10.  I then characterized each dimension by examining the items that had the highest/lowest scores on each one and giving a summary label (which was sometimes difficult, in which case I just used "unknown").  See run_mirt.R for details on how the model was run.





Now let's load in the models from each dimensionality (run on the cluster) and save their latent scores so that we can load them into python and have a look. (this is adapted from summarize_mirt.R)

In [12]:
datfiles=glob.glob('mirt_rdata/*Rdata')
compnums=[int(i.split('/')[1].replace('mirt_','').replace('dims_graded.Rdata','')) for i in datfiles]
compnums.sort()

if not os.path.exists('mirt_scores'):
    os.mkdir('mirt_scores')
    
%load_ext rpy2.ipython
%R require('mirt')

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


/Users/poldrack/anaconda/envs/py34/lib/python3.4/site-packages/rpy2/rinterface/__init__.py:185: RRuntimeWarning: Loading required package: mirt

  warnings.warn(x, RRuntimeWarning)
/Users/poldrack/anaconda/envs/py34/lib/python3.4/site-packages/rpy2/rinterface/__init__.py:185: RRuntimeWarning: Loading required package: stats4

  warnings.warn(x, RRuntimeWarning)
/Users/poldrack/anaconda/envs/py34/lib/python3.4/site-packages/rpy2/rinterface/__init__.py:185: RRuntimeWarning: Loading required package: lattice

  warnings.warn(x, RRuntimeWarning)


array([1], dtype=int32)

In [14]:
%%R -i compnums -o converged
#compnums=c(1,2,3,4,5,6,8,9,10)
converged=array(dim=length(compnums))
for (i in 1:length(compnums)) {
  ncomps=compnums[i]
  load(sprintf('mirt_rdata/mirt_%ddims_graded.Rdata',ncomps))
  converged[i]=m@OptimInfo$converged
  s=summary(m,verbose=FALSE)
  scores=s$rotF
  write.table(scores,file=sprintf('mirt_scores/mirt_scores_%ddims.tsv',ncomps),sep='\t',quote=FALSE,col.names=FALSE)
}

Plot some convergence statistics for each model - note that the higher-dimensional models have not converged to the default criterion (after 5000 MH-RM iterations).

In [16]:
# print out some convergence statistics for each model
if numpy.sum(converged==0)>0:
    df=pandas.DataFrame({'Dimensions':compnums,'converged':converged})
    display(df)
else:
    print('all models converged')

all models converged


Now let's look at the results from each model.   First we need to set up a dictionary with the subjective labels NB: These are manually generated using the results from below.

In [21]:
compdesc={}
compdesc[1]=['persistence']
compdesc[2]=['persistence','risktaking']
compdesc[3]=['persistence','physical risk','impulsivity']
compdesc[4]=['persistence','physical risk','impulsivity','internalizing']
compdesc[5]=['manic impulsivity','physical risk','impulsivity','internalizing',
            'persistence']
compdesc[6]=['impulsivity','physical risk','life impulsivity',
            'internalizing','persistence','planning/sentimentality']
compdesc[7]=['manic impulsivity','physical risk','dysfunctional risk',
             'internalizing','persistence','unknown','life impulsivity']
compdesc[8]=['manic impulsivity','physical risk','dysfunctional risk',
            'internalizing','persistence','unknown','life impulsivity',
            'mindfulness']
compdesc[9]=['manic impulsivity','physical risk','financial risk','internalizing',
             'unknown','life risk','persistence','impulsivity','mindfulness']
compdesc[10]=['manic impulsivity','physical risk','financial risk','internalizing',
              'mental energy','life risk','persistence','impulsivity','mindfulness','emotion regulation']

Now let's list the items with top positive and negative scores for each latent dimension, along with the subjective label.

In [22]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [75]:
scdata.head()

,itemname,score1,score2,score3,score4,score5,score6,score7,score8,score9,score10
0,X,-0.060671,0.007157,-0.028218,0.098587,-0.209969,-0.037062,-0.088019,0.129428,-0.001248,-0.082237
1,bis11_survey_2,0.035908,-0.386244,0.024182,-0.112665,0.059790,-0.104206,0.425424,-0.290514,-0.137073,0.045451
2,bis11_survey_3,-0.304449,-0.214627,0.023867,0.054022,0.055503,0.076143,0.183039,-0.410053,-0.228875,-0.084855
3,bis11_survey_4,-0.000989,-0.037438,0.087403,-0.013978,-0.187589,0.013685,-0.054886,-0.325237,0.161744,-0.146488
4,bis11_survey_5,-0.139359,-0.083245,-0.024445,-0.285582,0.153273,0.228069,-0.094359,-0.069197,0.059324,-0.345981


In [89]:

mirt_scores={}
for c in range(len(compnums)):
    ncomps=compnums[c]
    scdata=pandas.read_csv('mirt_scores/mirt_scores_%ddims.tsv'%ncomps,
                         delimiter='\t',header=None,names=['itemname']+['score%d'%int(i+1) for i in range(ncomps)])
    scdata=scdata[scdata.itemname != 'X']
    mirt_scores[ncomps]=scdata
    print('')
    for d in range(ncomps):
        print('%d Components: Dimension %d (%s)'%(ncomps,d+1,compdesc[ncomps][d]))
        cdata=scdata.sort_values(by='score%d'%int(d+1))
        lowdata=cdata.iloc[:3,[0,d+1]]
        highdata=cdata.iloc[-3:,[0,d+1]]
        data_show=pandas.concat([lowdata,highdata])
        itemtext=[]
        for item in data_show['itemname']:
            itemtext.append(all_metadata[item]['Description'])
        data_show['itemtext']=itemtext
        display(data_show)    


2 Components: Dimension 1 (persistence)


,itemname,score1,itemtext
456,upps_impulsivity_survey_10,-0.730868,I tend to give up easily.
451,upps_impulsivity_survey_5,-0.626464,I generally like to see things through to the end.
474,upps_impulsivity_survey_28,-0.609298,I finish what I start.
352,self_regulation_survey_14,0.612457,I am able to accomplish goals for myself.
263,grit_scale_survey_7,0.624460,I have difficulty maintaining my focus on projects that take more than a few months to complete.
350,self_regulation_survey_12,0.737828,I give up quickly.


2 Components: Dimension 2 (risktaking)


,itemname,score2,itemtext
268,impulsive_venture_survey_4,-0.846512,Do you quite enjoy taking risks?
277,impulsive_venture_survey_13,-0.835872,Do you usually think carefully before doing anything?
470,upps_impulsivity_survey_24,-0.787585,I quite enjoy taking risks.
327,mpq_control_survey_13,0.713378,I am cautious person.
329,mpq_control_survey_15,0.749584,I do the first thing that comes to mind.
318,mpq_control_survey_4,0.779842,I think before doing.



3 Components: Dimension 1 (persistence)


...

,itemname,score8,itemtext
275,impulsive_venture_survey_11,-0.779718,Are you an impulsive person?
399,time_perspective_survey_9,-0.698804,I do things impulsively.
279,impulsive_venture_survey_15,-0.675021,Do you often do things on the spur of the moment?
328,mpq_control_survey_14,0.495845,I am a spontaneous person.
324,mpq_control_survey_10,0.516521,I do things on the spur of the moment.
329,mpq_control_survey_15,0.651321,I do the first thing that comes to mind.


10 Components: Dimension 9 (mindfulness)


,itemname,score9,itemtext
294,impulsive_venture_survey_30,-0.529667,Do you often change your interests?
5,bis11_survey_6,-0.509302,I don't pay attention.
484,upps_impulsivity_survey_38,-0.481322,I am a person who always gets the job done.
308,mindful_attention_awareness_survey_9,0.844982,I rush through activities without being really attentive to them.
307,mindful_attention_awareness_survey_8,0.851730,It seems I am 'running on automatic' without much awareness of what Im doing.
221,five_facet_mindfulness_survey_14,0.853365,I am easily distracted.


10 Components: Dimension 10 (emotion regulation)


,itemname,score10,itemtext
245,five_facet_mindfulness_survey_38,-0.761723,I can usually describe how I feel at the moment in considerable detail.
210,five_facet_mindfulness_survey_3,-0.746166,Im good at finding words to describe my feelings.
240,five_facet_mindfulness_survey_33,-0.738333,My natural tendency is to put my experiences into words.
200,erq_survey_3,0.412284,I keep my emotions to myself.
202,erq_survey_5,0.466388,"When I am feeling <i>positive</i> emotions, I am careful not to express them."
339,mpq_control_survey_25,0.605738,I like to find out what to expect.


I then analyzed how the dimensions related across each level of complexity, and created a graph showing how the different components related, which is attached here.  the labels of each node reflect the level of dimensionality, the number of the dimension, and my description of the items associated with the dimension.  the links in the graph are based on the strongest correlation between dimensions across levels. what is very cool is that there is a very orderly relation between dimensions at each level. I save the file out to a GraphML file that I can import into Cytoscape for visualization, as the standard methods in networkx don't do a great job of displaying the graph.



In [24]:
corrs={}
g=nx.DiGraph()
verbose=False

scores={}
for ncomps in compnums:
    scores[ncomps]=pandas.read_csv('mirt_scores/mirt_scores_%ddims.tsv'%ncomps,
                        delimiter='\t',header=None)

# first set up all nodes
for i in range(len(compnums)):
    for j in range(compnums[i]):
        xnode='%d-%d'%(compnums[i],j+1)
        assert not xnode in g.nodes()
        g.add_node(xnode)
        g.node[xnode]['level']=compnums[i]
        g.node[xnode]['desc']=xnode+':'+compdesc[compnums[i]][j]
        if verbose:
            print('added',xnode,g.node[xnode])

# find strong edges between nodes at each level
for i in range(len(compnums)-1):
    s1=scores[compnums[i]]    

    for j in range(i+1,i+2):
        s2=scores[compnums[j]]
        corrs[(i+1,j+1)]=numpy.zeros((compnums[i],compnums[j]))
        for x in range(compnums[i]):
            xnode='%d-%d'%(compnums[i],x+1)
            for y in range(compnums[j]):
                ynode='%d-%d'%(compnums[j],y+1)
                corrs[(i+1,j+1)][x,y]=numpy.corrcoef(s1.loc[:,x+1],s2.loc[:,y+1])[0,1]
            matchnum=numpy.argmax(numpy.abs(corrs[(i+1,j+1)][x,:]))
            mnode='%d-%x'%(compnums[j],matchnum+1)
            g.add_edge(xnode,'%d-%x'%(compnums[j],matchnum+1))
            g[xnode]['%d-%x'%(compnums[j],matchnum+1)]['weight']=float(corrs[(i+1,j+1)][x,matchnum])


            if verbose:
                print('%d-comp%d: matches %d-comp%d (%0.3f)'%(compnums[i],
                    x+1,compnums[j],matchnum+1,corrs[(i+1,j+1)][x,matchnum]))
                


# for nodes with no inputs, find  largest input and add link
if 1:
    indegree=g.in_degree()
    for k in indegree.keys():
        if indegree[k]>0 or k.split('-')[0]=='1':
            continue
        i,j=[int(x) for x in k.split('-')]
        # get correlations with previous level
        c=corrs[(i-1,i)]
        offset=2
        crow=c[:,i-1]
        matchnum=numpy.argmax(numpy.abs(crow))
        newedge='%d-%d'%(compnums[i-offset],matchnum+1)
        g.add_edge(newedge,k)
        g[newedge][k]['weight']=float(crow[matchnum])
        if verbose:
            print(k,i,j,matchnum,newedge,crow)


nx.write_graphml(g,'relations.graphml')

KeyError: (9, 10)

And here is the nicer version created by hand using Cytoscape.

In [ ]:
showPDF('https://dl.dropboxusercontent.com/u/2441264/mirt_graph.pdf')

### IRT via exploratory factor analysis

These analyses use the irt.fa function from the R psych package to perform exploratory factor analysis.  

In [50]:
%%R -o indices
indices=matrix(nrow=10,ncol=2)

for (ncomps in 1:10) {
  load(sprintf('irtfa_rdata/irtfa_%ddims_corcor.Rdata',ncomps))
  indices[ncomps,1]=irtmodel$fa$RMSEA[1]
  indices[ncomps,2]=irtmodel$fa$SABIC
  loadings=irtmodel$fa$loadings
  write.table(loadings,file=sprintf('irtfa_scores/irtfa_scores_%ddims.tsv',ncomps),sep='\t',quote=FALSE,col.names=FALSE)
}

In [51]:
showtable=False
if numpy.max(indices[:,0])>0.1 or showtable:
    print('Check for bad fit')
    df=pandas.DataFrame({'Dimensions':numpy.arange(1,11),'RMSEA':indices[:,0],'SABIC':indices[:,1]})
    display(df)
else:
    print('all RMSEA values < 0.1')


all RMSEA values < 0.1


In [ ]:
irtfa_compdesc={}
irtfa_compdesc[1]=['manic impulsivity']
irtfa_compdesc[2]=['persistence','risktaking']
irtfa_compdesc[3]=['persistence','impulsivity','physical risk']
irtfa_compdesc[4]=['persistence','impulsivity','physical risk','unclear']
irtfa_compdesc[5]=['impulsivity','physical risk','persistence','internalizing',
            'life risk']
irtfa_compdesc[6]=['impulsivity','physical risk','life impulsivity',
            'internalizing','persistence','planning/sentimentality']
irtfa_compdesc[7]=['manic impulsivity','physical risk','dysfunctional risk',
             'internalizing','persistence','unknown','life impulsivity']
irtfa_compdesc[8]=['manic impulsivity','physical risk','dysfunctional risk',
            'internalizing','persistence','unknown','life impulsivity',
            'mindfulness']
irtfa_compdesc[9]=['manic impulsivity','physical risk','financial risk','internalizing',
             'unknown','life risk','persistence','impulsivity','mindfulness']
irtfa_compdesc[10]=['manic impulsivity','physical risk','financial risk','internalizing',
              'mental energy','life risk','persistence','impulsivity','mindfulness','emotion regulation']

In [52]:


irtfa_scores={}
for c in range(1,11):
    ncomps=c
    scdata=pandas.read_csv('irtfa_scores/irtfa_scores_%ddims.tsv'%ncomps,
                         delimiter='\t',header=None,names=['itemname']+['score%d'%int(i+1) for i in range(ncomps)])
    irtfa_scores[ncomps]=scdata
    print('')
    for d in range(ncomps):
        print('%d Dimensions: Dimension %d (TBD)'%(ncomps,d+1))
#        print('%d Components: Dimension %d (%s)'%(ncomps,d+1,compdesc[ncomps][d]))
        cdata=scdata.sort_values(by='score%d'%int(d+1))
        lowdata=cdata.iloc[:3,[0,d+1]]
        highdata=cdata.iloc[-3:,[0,d+1]]
        data_show=pandas.concat([lowdata,highdata])
        itemtext=[]
        for item in data_show['itemname']:
            itemtext.append(all_metadata[item]['Description'])
        data_show['itemtext']=itemtext
        display(data_show)    


1 Dimensions: Dimension 1 (TBD)


,itemname,score1,itemtext
2,ten_item_personality_survey_10,-0.439740,"Disorganized, careless."
5,self_regulation_survey_11,-0.418575,I learn from my mistakes.
6,brief_self_control_survey_14,-0.417791,I often act without thinking through all the alternatives.
3,upps_impulsivity_survey_50,0.433350,"When I am really excited, I tend not to think of the consequences of my actions."
1,upps_impulsivity_survey_31,0.444849,Others would say I make bad choices when I am extremely happy about something.
0,upps_impulsivity_survey_16,0.463326,"When I am very happy, I tend to do things that may cause problems in my life."



2 Dimensions: Dimension 1 (TBD)


,itemname,score1,itemtext
0,upps_impulsivity_survey_10,-0.459776,I tend to give up easily.
2,upps_impulsivity_survey_28,-0.442061,I finish what I start.
3,upps_impulsivity_survey_48,-0.434633,Sometimes there are so many little things to be done that I just ignore them all.
5,self_regulation_survey_14,0.426022,I am able to accomplish goals for myself.
4,mindful_attention_awareness_survey_8,0.431182,It seems I am 'running on automatic' without much awareness of what Im doing.
1,grit_scale_survey_7,0.457339,I have difficulty maintaining my focus on projects that take more than a few months to complete.


2 Dimensions: Dimension 2 (TBD)


...

,itemname,score8,itemtext
424,upps_impulsivity_survey_22,-0.257267,I don't like to start a project until I know exactly how to proceed.
346,erq_survey_5,-0.249345,"When I am feeling <i>positive</i> emotions, I am careful not to express them."
325,impulsive_venture_survey_14,-0.242624,Would you like to learn to fly an aeroplane?
415,bis_bas_survey_15,0.409164,When I see an opportunity for something I like I get excited right away.
414,bis_bas_survey_10,0.458996,When I want something I usually go all-out to get it.
413,bis_bas_survey_19,0.468834,"When good things happen to me, it affects me strongly."


10 Dimensions: Dimension 9 (TBD)


,itemname,score9,itemtext
448,dospert_rt_survey_8,-0.373990,Disagreeing with an authority figure on a major issue.
449,dospert_rt_survey_29,-0.369722,Starting a new career in your mid-thirties.
454,dospert_rt_survey_22,-0.333715,Choosing a career that you truly enjoy over a more secure one.
452,dospert_rp_survey_23,0.343551,Speaking your mind about an unpopular issue in a meeting at work.
451,dospert_rp_survey_28,0.346451,Moving to a city far away from your extended family.
450,dospert_rp_survey_8,0.362789,Disagreeing with an authority figure on a major issue.


10 Dimensions: Dimension 10 (TBD)


,itemname,score10,itemtext
479,dickman_survey_13,-0.333980,"I don't like to do things quickly, even when I am doing something that is not very difficult."
483,dickman_survey_16,-0.278287,"I like to take part in really fast-paced conversations, where you don't have much time to think ..."
485,mpq_control_survey_24,-0.255525,I have a good idea of what I'm going to do.
478,bis11_survey_16,0.354258,I like to think about complex problems.
477,theories_of_willpower_survey_7,0.359645,"After a strenuous mental activity, you feel energized for further challenging activities."
476,time_perspective_survey_30,0.418059,I get nostalgic about my childhood.


### compare MIRT and IRTFA results

In [111]:
for ncomps in range(2,11):
    isc=irtfa_scores[ncomps].sort_values(by='itemname')
    msc=mirt_scores[ncomps].sort_values(by='itemname')
    msc=msc.loc[msc['itemname'].isin(list(isc['itemname']))]
    assert all(msc.itemname==isc.itemname)
    for i in range(ncomps):
        irtfadata=isc.iloc[:,i+1]

        for j in range(ncomps):
            mirtdata=msc.iloc[:,j+1]
            print(ncomps,i+1,j+1,numpy.corrcoef(irtfadata,mirtdata)[0,1])

2 1 1 0.965053981749
2 1 2 0.244193307513
2 2 1 0.0141087774522
2 2 2 -0.944602010004
3 1 1 0.896591890282
3 1 2 -0.266809952177
3 1 3 0.408429553225
3 2 1 -0.00900184936955
3 2 2 -0.314109947603
3 2 3 -0.867049358695
3 3 1 -0.371390021019
3 3 2 -0.830705897952
3 3 3 0.0398953736141
4 1 1 0.802696724028
4 1 2 -0.124948209253
4 1 3 0.414493723632
4 1 4 -0.494408509151
4 2 1 -0.171117561408
4 2 2 -0.271826549918
4 2 3 -0.860750765804
4 2 4 0.0355340974956
4 3 1 -0.539613735362
4 3 2 -0.683359698036
4 3 3 -0.028965766154
4 3 4 0.101916828601
4 4 1 -0.334616009379
4 4 2 0.466036197729
4 4 3 -0.32741546756
4 4 4 0.652225092539
5 1 1 -0.730497450051
5 1 2 -0.345151006294
5 1 3 -0.435813844941
5 1 4 -0.145448900597
5 1 5 0.472687437414
5 2 1 0.283524698145
5 2 2 0.0265413066273
5 2 3 -0.216700968604
5 2 4 -0.400203638952
5 2 5 -0.67633513693
5 3 1 -0.0812808979919
5 3 2 -0.737709324919
5 3 3 0.459071924964
5 3 4 0.23249967846
5 3 5 0.296687020412
5 4 1 -0.225699011889
5 4 2 0.320660842739
5 4

In [108]:
isc=irtfa_scores[ncomps].sort_values(by='itemname')

### IClust 
Run a similar analysis using iclust from the R Psych package (based on Revelle, 1979)

In [ ]:
surveydata=pandas.read_csv('surveydata.csv',index_col='worker')
varnames=list(surveydata.columns)
surveydata.shape
len(varnames)

In [ ]:
%%R -i surveydata,varnames
library(psych)
df=data.frame(surveydata)
names(df)=varnames
pdf(file='iclust_surveys.pdf',width=16, height=16)
iclust(df)
dev.off()

### Cross-validation analyses

In parallel to these I have also run a set of analyses where I perform cross-validation by estimating the model on split halves of the data and then compute the log-likelihood of the held-out data given the model; this is a standard method in machine learning for estimating the appropriate model order in an unbiased way.  

First, we load the output from the crossvalidation analyses that were run on the TACC Wrangler cluster (using run_mirt_cv.R).  There was a complication that arose due to the fact that mirt() works on a full data matrix that expands each item into all possible response values; if there is a response on a particular item that occurs in one dataset but not in another, then this causes a mismatch between models which makes it impossible to assess the likelihood on the held-out data.  My solution to this was to find responses that occurred rarely, and either move them towards the center of the scale (e.g., if there were a small number of "7" responses on a 1-7 scale, change them to "6" responses), or to delete the item (if there were more than two low-frequency responses, or if they occurred in a non-extreme position).  This was performed using cleanup_items_for_mirt_cv.py; the analyses presented here used a minimum response frequency of 8, which resulted in removal of 23 of the 507 original items.  Thus, the data going into the crossvalidation are not exactly the same as the full data analyzed above.


In [ ]:
meanll=numpy.zeros(10)
dimvals=[]
for i in range(1,10):
    files=glob.glob('cvdata/ll_%03d_*'%i)
    if len(files)==0:
        continue
    m=[]
    dimvals.append(i)
    for f in files:
        d=numpy.loadtxt(f)
        m.append(numpy.mean(d))
    meanll[i]=numpy.mean(m)

plt.plot(dimvals,meanll[1:])
plt.xlabel('Number of dimensions')
plt.ylabel('log-likeihood of held-out data')
print('Crossvalidation suggests %d dimensions'%dimvals[numpy.argmax(meanll[1:])])